In [20]:
import requests

payload = {
    'httpAccept': 'application/json',
    'queryType': 'searchTerms',
    'query': '+'.join(['climate', 'change'])
}
url = 'http://api.nature.com/content/opensearch/request'
result = requests.get(url, params=payload)
response = result.json()


In [23]:
import pyperclip, webbrowser

def browseJSON(json_str):
    pyperclip.copy(json_str)
    webbrowser.open_new_tab('https://jsonformatter.curiousconcept.com/')
    
browseJSON(result.text)

# then paste in your clipboard and set the JSON type away from default